### 유사도를 통한 콘텐츠 기반의 추천 시스템과 군집화 분석

In [ ]:
# !pip install -Uq koreanize-matplotlib

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import seaborn as sns
import koreanize_matplotlib
import matplotlib.pyplot as plt

conn = sqlite3.connect('yes24_books_it.db')
df = pd.read_sql_query("SELECT * FROM books", conn)
conn.close()

In [ ]:
df

## 유사도 계산

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 데이터 전처리
df['combined_text'] = df['title'] + ' ' + df['description'].fillna('')

# TF-IDF 벡터화
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['combined_text'])
tfidf_matrix

In [ ]:
vectorizer.get_feature_names_out()

In [ ]:
df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
df_tfidf.head()

In [ ]:
df_tfidf.sum().nlargest(50).plot(kind="bar", figsize=(12, 4))

In [ ]:
# 코사인 유사도 계산
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim

In [ ]:
sns.heatmap(cosine_sim[:100, :100])

## 유사도 정렬로 콘텐츠 기반 추천시스템 작성

In [ ]:
# 유사도 기반 책 추천 함수
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = df[df['title'].str.contains(title)].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]  # 상위 5개 추천
    book_indices = [i[0] for i in sim_scores]
    return df['title'].iloc[book_indices]

# 제목으로 유사한 책 추천
get_recommendations('프롬프트 엔지니어링')

In [ ]:
get_recommendations('생성형AI')

### 콘텐츠 기반 추천 시스템(Content-based Recommender System)

In [ ]:
# 문서 번호로 유사도를 찾는 함수
def get_recommendations_by_index(index, cosine_sim=cosine_sim, df=df):
    sim_scores = list(enumerate(cosine_sim[index]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]  # 상위 5개 추천 (자기 자신 제외)
    book_indices = [i[0] for i in sim_scores]
    
    result = df.iloc[book_indices][['title', 'author', 'publisher']]
    result['similarity'] = [i[1] for i in sim_scores]
    
    return result

In [ ]:
# 사용 예시
index = 0  # 첫 번째 책의 인덱스
similar_books = get_recommendations_by_index(index)
print(f"책 '{df.iloc[index]['title']}'와 유사한 책들:")
display(similar_books)

# 랜덤한 인덱스로 테스트
random_index_list = np.random.randint(0, len(df), 3)
for random_index in random_index_list:
    similar_books = get_recommendations_by_index(random_index)
    print(f"\n책 '{df.iloc[random_index]['title']}'와 유사한 책들:")
    display(similar_books)

### 군집화 

In [ ]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from collections import Counter

In [ ]:
tfidf_matrix

In [ ]:
# K-means 군집화
num_clusters = 10  # 클러스터 수 지정
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
df['cluster'] = kmeans.fit_predict(tfidf_matrix)

# PCA를 사용한 차원 축소 (시각화를 위해)
pca = PCA(n_components=2)
pca_result = pca.fit_transform(tfidf_matrix.toarray())
df['pca_x'] = pca_result[:, 0]
df['pca_y'] = pca_result[:, 1]

# 시각화
plt.figure(figsize=(12, 8))
sns.scatterplot(data=df, x='pca_x', y='pca_y', hue='cluster', palette='deep')
plt.title('Book Clusters Visualization')
plt.show()

In [ ]:
# 각 클러스터의 대표적인 단어 추출
def get_top_words(cluster):
    text = ' '.join(df[df['cluster'] == cluster]['combined_text'])
    words = text.split()
    word_counts = Counter(words)
    return [word for word, count in word_counts.most_common(10) if len(word) > 2]


def get_top_tfidf_words(cluster):
    text = ' '.join(df[df['cluster'] == cluster]['combined_text'])
    cluster_idx = df[df['cluster'] == cluster].index
    return df_tfidf.loc[cluster_idx].sum().nlargest(10).index.tolist()

# 클러스터별 분석
for i in range(num_clusters):
    cluster_books = df[df['cluster'] == i]
    print(f"\nCluster {i}:")
    print(f"Number of books: {len(cluster_books)}")
    print(f"Top words: {get_top_words(i)}")
    print(f"Top tf-idf words: {get_top_tfidf_words(i)}")
    print("Sample titles:")
    print(cluster_books['title'].head().to_string(index=False))
    print("-" * 50)